In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)

In [6]:
lyrics_len = 1301

In [7]:
dataset = pd.read_csv('data/train_{}.csv'.format(lyrics_len))

In [8]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,26,74,54,68,1,69,64,1,69,50,...,70,1,72,64,70,61,53,63,4,69
1,30,69,4,68,1,50,1,13,16,10,...,1,64,63,1,51,50,53,1,69,54
2,24,74,67,70,68,1,31,64,63,54,...,1,67,50,58,63,1,1,0,28,67
3,29,54,61,61,64,7,1,1,0,30,...,57,54,67,54,1,58,68,1,63,64
4,26,61,54,50,63,64,67,1,74,64,...,64,70,67,1,54,74,54,68,1,72


In [9]:
dataset.shape

(46046, 301)

In [10]:
batchs = pd.read_csv('data/batch_sizes.csv', index_col=0, header=None)

In [11]:
batchs.head()

,1
0,
848,79
734,78
739,75
850,74
860,74


In [12]:
emb_dim = 128
input_length = 300
batch_size = batchs.loc[lyrics_len, 1]

In [13]:
batch_size

46

In [14]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (46, 300, 128)            9728      
_________________________________________________________________
conv1d_1 (Conv1D)            (46, 300, 256)            131328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (46, 150, 256)            0         
_________________________________________________________________
dropout_1 (Dropout)          (46, 150, 256)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (46, 150, 32)             32800     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (46, 75, 32)              0         
_________________________________________________________________
bidirectional_1 (Bidirection (46, 75, 200)             106400    
__________

In [16]:
model.set_weights(load_model('model_300.h5').get_weights())

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
for _ in range(5):
    print(_)
    model.reset_states()
    model.fit(dataset.iloc[:, :-1].values, dataset.iloc[:, [-1]].values, epochs=1, batch_size=batch_size)

0
Epoch 1/1
46046/46046 [==============================] - 168s - loss: 1.6825 - acc: 0.5272   
1
Epoch 1/1
46046/46046 [==============================] - 167s - loss: 1.5733 - acc: 0.5489   
2
Epoch 1/1
46046/46046 [==============================] - 167s - loss: 1.5132 - acc: 0.5632   
3
Epoch 1/1
46046/46046 [==============================] - 167s - loss: 1.4873 - acc: 0.5693   
4
Epoch 1/1
46046/46046 [==============================] - 167s - loss: 1.4541 - acc: 0.5766   


In [19]:
# save model
model.save('model_300.h5')

In [20]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dense(vocab_size, activation='softmax'))

In [21]:
pred_model.set_weights(model.get_weights())

In [22]:
def implement(seed_text, maxlen=100, must_stop=2000, n_likely=5):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-n_likely:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [29]:
res = implement('We are friends, good friends', 300, 200, 20)

In [30]:
print(res)

We are friends, good friends,  
Gu-nwin the reide

(end)  
Just  
She ex-Esle days out  
When everydom a oof glag  
A e-to ney rere sumring make care say  
Word all my thafe  
When the breaed and a dream dering  
Chesin' siad, h
